In [1]:
import pandas as pd
import numpy as np
import os

# Question 1

In [4]:
path = "/Users/yifuhe/Learning/RBS/fin_model/homework6"
HXZ_factor = pd.read_excel(path+"/Data_q_factors.xlsx")
five_factor = pd.read_excel(path+"/F-F_Research_Data_5_Factors_2x3.xlsx")
fama_french = pd.read_excel(path+"/F-F_Research_Data_Factors.xlsx")
# read data from path
port_25 = pd.read_excel(path+"/25_Portfolios_5x5.xlsx")
# 25 Fama-French size and book to market sorted portfolios 
colname = port_25.iloc[14,:].values
colname[0] = "Date"
data = port_25.iloc[15:-2,:].reset_index().drop("index",axis = 1)
data.columns = colname
for i in range(len(data["Date"])):
    if not isinstance(data["Date"][i],(int,float)):
        print(i,data.Date[i])
#data["Date"] = pd.to_datetime(data.loc["Date"].apply(str),format = "%Y%m")



1126   Average Equal Weighted Returns -- Monthly
2254   Average Value Weighted Returns -- Annual
2351   Average Equal Weighted Returns -- Annual
2448   Number of Firms in Portfolios
3576   Average Market Cap
4704   For portfolios formed in June of year t
4705   Value Weight Average of BE/ME Calculated for June of t to June of t+1 as: 
4706   Sum[ME(Mth) * BE(Fiscal Year t-1) / ME(Dec t-1)] / Sum[ME(Mth)]
4707   Where Mth is a month from June of t to June of t+1
4708   and BE(Fiscal Year t-1) is adjusted for net stock issuance to Dec t-1
5836   For portfolios formed in June of year t
5837   Value Weight Average of BE_FYt-1/ME_June t Calculated for June of t to June of t+1 as: 
5838   Sum[ME(Mth) * BE(Fiscal Year t-1) / ME(Jun t)] / Sum[ME(Mth)]
5839   Where Mth is a month from June of t to June of t+1
5840   and BE(Fiscal Year t-1) is adjusted for net stock issuance to Jun t
6968   For portfolios formed in June of year t
6969   Value Weight Average of OP Calculated as: 
6970   Sum[ME(Mt

In [7]:
industry_30 = pd.read_excel(path+"/30_Industry_Portfolios.xlsx")
for i in range(len(industry_30.iloc[:,0])):
    if not isinstance(industry_30.iloc[i,0],(int,float)):
        print(i,industry_30.iloc[i,0])

colname = industry_30.iloc[10,:].values
colname[0] = "Date"
AVWR_month = industry_30.iloc[11:1135].reset_index().drop("index",axis=1)
AVWR_month.columns = colname
AVWR_month["Date"] = pd.to_datetime(AVWR_month["Date"],format = "%Y%m")
AVWR_month.head()

0 It contains value- and equal-weighted returns for 30 industry portfolios.
2 The portfolios are constructed at the end of June.
4 The annual returns are from January to December.
6 Missing data are indicated by -99.99 or -999.
9   Average Value Weighted Returns -- Monthly
1137   Average Equal Weighted Returns -- Monthly
2265   Average Value Weighted Returns -- Annual
2362   Average Equal Weighted Returns -- Annual
2459   Number of Firms in Portfolios
3587   Average Firm Size
4715   Sum of BE / Sum of ME
4813   Value-Weighted Average of BE/ME
4910 Copyright 2020 Kenneth R. French


,Date,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
0,1926-07-01,0.56,-5.19,1.29,2.93,10.97,-0.48,8.08,1.77,8.14,...,0.83,9.22,2.06,7.7,1.93,-23.79,0.07,1.87,0.37,5.2
1,1926-08-01,2.59,27.03,6.5,0.55,10.01,-3.58,-2.51,4.25,5.5,...,2.17,2.02,4.39,-2.38,4.88,5.39,-0.75,-0.13,4.46,6.76
2,1926-09-01,1.16,4.02,1.26,6.58,-0.99,0.73,-0.51,0.69,5.33,...,2.41,2.25,0.19,-5.54,0.05,-7.87,0.25,-0.56,-1.23,-3.86
3,1926-10-01,-3.06,-3.31,1.06,-4.76,9.47,-4.68,0.12,-0.57,-4.76,...,-0.11,-2,-1.09,-5.08,-2.64,-15.38,-2.2,-4.11,-5.16,-8.49
4,1926-11-01,6.35,7.29,4.55,1.66,-5.8,-0.54,1.87,5.42,5.2,...,1.63,3.77,3.64,3.84,1.6,4.67,6.52,4.33,2.24,4


In [8]:
def get_portfolio_pool(data, J, decile=3):
    res = []
    cumulate = data.rolling(J).sum().dropna()
    slices = [i for i in range(decile)] + [i for i in range(-decile,0)]
    index = cumulate.index
    for i in index:
        sorted_slice = cumulate.sort_values(by = i ,axis = 1, ascending = False)
        res.append(sorted_slice.columns[slices])
    colname = ["long_{}".format(i) for i in range(decile)] + ["short_{}".format(i) for i in range(decile,0,-1)]
    return pd.DataFrame(res,columns=colname,index=index)

portfolio_pool_6 = get_portfolio_pool(AVWR_month,6,3)
portfolio_pool_9 = get_portfolio_pool(AVWR_month,9,3)
portfolio_pool_6.head()

,long_0,long_1,long_2,short_3,short_2,short_1
5,Autos,Beer,Chems,Hshld,Paper,Whlsl
6,Beer,Books,Txtls,Other,Paper,Whlsl
7,Servs,Chems,Txtls,Paper,Other,Whlsl
8,Chems,Mines,BusEq,Other,Paper,Whlsl
9,Chems,Mines,Autos,Paper,Oil,Whlsl


In [93]:
def long_short(month_return, portfolio_pool, K, skip=1):
    index, end = portfolio_pool.index[1+skip:], portfolio_pool.index[-1]
    raw_df = pd.DataFrame(np.zeros((len(index),3)),index=index,columns=["long","short","arbitrage"])
    for i in index:
        hold_index = list(range(i,min(end,i+K)))
        for i in hold_index:
            long = portfolio_pool.loc[i,["long_0","long_1","long_2"]].values
            short = portfolio_pool.loc[i,["short_3","short_2","short_1"]].values
            #print(company_columns)
            raw_df.loc[i,"long"] += month_return.loc[i,long].sum()
            raw_df.loc[i,"short"] += month_return.loc[i,short].sum()
    raw_df["long"] /= 18
    raw_df["short"] /= 18
    raw_df["arbitrage"] = raw_df["long"] - raw_df["short"] 
    return raw_df

p66 = long_short(AVWR_month,portfolio_pool_6,6)
p69 = long_short(AVWR_month,portfolio_pool_6,9)
p96 = long_short(AVWR_month,portfolio_pool_9,6)
p99 = long_short(AVWR_month,portfolio_pool_9,9)

Because of the t-value is greater than the 95% tscore, all the excess return are significant.

In [97]:
print(f"df: {p66.shape[0]}")
print(f"95%t-score: {1.646}")
print(f't66: {p66["arbitrage"].mean()/p66["arbitrage"].std()*np.sqrt(p66.shape[0])}')
print(f't69: {p69["arbitrage"].mean()/p69["arbitrage"].std()*np.sqrt(p66.shape[0])}')
print(f't96: {p96["arbitrage"].mean()/p96["arbitrage"].std()*np.sqrt(p66.shape[0])}')
print(f't99: {p99["arbitrage"].mean()/p99["arbitrage"].std()*np.sqrt(p66.shape[0])}')

df: 1117
95%t-score: 1.646
t66: 30.830085692495242
t69: 30.7784947528333
t96: 24.879829686348785
t99: 24.847067914443098


In [133]:
# get rf
temp_data = pd.read_excel(path+"/F-F_Research_Data_Factors.xlsx")
rf = temp_data.iloc[3:1127,4].mean()
Rm = temp_data.iloc[3:1127,2]
rf *=10
print(f"rf: {rf}")

# compute sharpe ratio
print(f'sharpe ratio 66: {(p66["long"].mean() - rf)/p66["long"].std()}')
print(f'sharpe ratio 69: {(p69["long"].mean() - rf)/p69["long"].std()}')
print(f'sharpe ratio 96: {(p96["long"].mean() - rf)/p96["long"].std()}')
print(f'sharpe ratio 99: {(p99["long"].mean() - rf)/p99["long"].std()}')
print(f'market portfolio sharpe ratio: {Rm.mean()/Rm.std()}')
print("All the sharpe ratio of long portfolio are higher than market portfolio")

rf: 2.7283807829181446
sharpe ratio 66: 0.15099621770830926
sharpe ratio 69: 0.2752308516025211
sharpe ratio 96: 0.0839389006728528
sharpe ratio 99: 0.21009221254283436
market portfolio sharpe ratio: 0.061386904331229016
All the sharpe ratio of long portfolio are higher than market portfolio


# Question 2

In [257]:
# import model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

# load data

factor_3 = pd.read_excel(path+"/F-F_Research_Data_Factors.xlsx")
name = [factor_3.iloc[2,1]]
CAPM = pd.DataFrame(factor_3.iloc[3:1127,1].values,columns=name)

name = factor_3.iloc[2,:].to_list()
F3 = pd.DataFrame(factor_3.iloc[3:1127,:].values,columns=name)
factor_3.columns = name


factor_5 = pd.read_excel(path+"/F-F_Research_Data_5_Factors_2x3.xlsx")
name = factor_5.iloc[2,:].to_list()
F5 = pd.DataFrame(factor_5.iloc[3:683,:].values,columns=name)

Q = pd.read_excel(path+"/Data_q_factors.xlsx")
Q["Rm-Rf"] = Q["R_MKT"]-Q["R_F"]
name = ["Rm-Rf","R_ME","R_IA","R_ROE",]
Q = pd.DataFrame(Q.loc[2:635,name].values,columns=name)
Q

,Rm-Rf,R_ME,R_IA,R_ROE
0,3.5998,1.9517,-1.6933,1.8876
1,3.4921,-0.7446,-2.9519,1.0983
2,-4.5545,2.9132,2.4686,0.5234
3,2.1555,6.2350,-2.1700,0.2945
4,4.2883,3.0119,2.3713,-0.7125
...,...,...,...,...
629,-2.4087,-4.2469,-0.9351,1.7673
630,1.2450,1.3935,3.0078,1.6828
631,1.6203,0.8401,-0.5515,0.6286
632,3.2577,-0.0261,-1.0127,-1.1667


## 66
Almost all the excess return are sinificant according to the p-value

In [328]:
# imports
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


# 66 CAPM
length = np.min([p66.shape[0],CAPM.shape[0]])
y = p66["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = CAPM.iloc[-length:,:].reset_index().drop("index",axis=1)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 CAPM")
print(lm1.summary())


# 66 F3
length = np.min([p66.shape[0],F3.shape[0]])
y = p66["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F3.iloc[-length:,1:4].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F3")
print(lm1.summary())

# 66 F5
length = np.min([p66.shape[0],F5.shape[0]])
y = p66["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F5")
print(lm1.summary())


# 66 Q
length = np.min([p66.shape[0],Q.shape[0]])
y = p66["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 Q")
print(lm1.summary())


# 66 CAPM
                                 OLS Regression Results                                
Dep. Variable:              arbitrage   R-squared (uncentered):                   0.018
Model:                            OLS   Adj. R-squared (uncentered):              0.017
Method:                 Least Squares   F-statistic:                              19.90
Date:                Tue, 12 May 2020   Prob (F-statistic):                    9.00e-06
Time:                        22:32:28   Log-Likelihood:                         -3925.6
No. Observations:                1117   AIC:                                      7853.
Df Residuals:                    1116   BIC:                                      7858.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

## 69
Almost all the excess return are sinificant according to the p-value

In [330]:
# imports
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


# 69 CAPM
length = np.min([p96.shape[0],CAPM.shape[0]])
y = p69["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = CAPM.iloc[-length:,:].reset_index().drop("index",axis=1)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 CAPM")
print(lm1.summary())


# 69 F3
length = np.min([p96.shape[0],F3.shape[0]])
y = p69["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F3.iloc[-length:,1:4].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F3")
print(lm1.summary())

# 69 F5
length = np.min([p96.shape[0],F5.shape[0]])
y = p69["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F5")
print(lm1.summary())


# 69 Q
length = np.min([p96.shape[0],Q.shape[0]])
y = p69["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 Q")
print(lm1.summary())


# 66 CAPM
                                 OLS Regression Results                                
Dep. Variable:              arbitrage   R-squared (uncentered):                   0.017
Model:                            OLS   Adj. R-squared (uncentered):              0.017
Method:                 Least Squares   F-statistic:                              19.77
Date:                Tue, 12 May 2020   Prob (F-statistic):                    9.61e-06
Time:                        22:33:47   Log-Likelihood:                         -4377.2
No. Observations:                1117   AIC:                                      8756.
Df Residuals:                    1116   BIC:                                      8762.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

## 96
Almost all the excess return are sinificant according to the p-value.
Except Rm-Rf in all the model, and SMB in the last one

In [332]:
# imports
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


# 96 CAPM
length = np.min([p96.shape[0],CAPM.shape[0]])
y = p96["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = CAPM.iloc[-length:,:].reset_index().drop("index",axis=1)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 CAPM")
print(lm1.summary())


# 96 F3
length = np.min([p96.shape[0],F3.shape[0]])
y = p96["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F3.iloc[-length:,1:4].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F3")
print(lm1.summary())

# 69 F5
length = np.min([p96.shape[0],F5.shape[0]])
y = p96["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 F5")
print(lm1.summary())


# 96 Q
length = np.min([p96.shape[0],Q.shape[0]])
y = p96["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 66 Q")
print(lm1.summary())


# 66 CAPM
                                 OLS Regression Results                                
Dep. Variable:              arbitrage   R-squared (uncentered):                   0.010
Model:                            OLS   Adj. R-squared (uncentered):              0.009
Method:                 Least Squares   F-statistic:                              11.13
Date:                Tue, 12 May 2020   Prob (F-statistic):                    0.000878
Time:                        22:34:54   Log-Likelihood:                         -3835.7
No. Observations:                1114   AIC:                                      7673.
Df Residuals:                    1113   BIC:                                      7678.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

## 99
Almost all the excess return are sinificant according to the p-value.
except the Rm-Rf in all the model

In [333]:
# imports
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


# 99 CAPM
length = np.min([p99.shape[0],CAPM.shape[0]])
y = p99["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = CAPM.iloc[-length:,:].reset_index().drop("index",axis=1)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 99 CAPM")
print(lm1.summary())


# 99 F3
length = np.min([p99.shape[0],F3.shape[0]])
y = p99["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F3.iloc[-length:,1:4].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 99 F3")
print(lm1.summary())

# 99 F5
length = np.min([p99.shape[0],F5.shape[0]])
y = p99["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 99 F5")
print(lm1.summary())


# 99 Q
length = np.min([p99.shape[0],Q.shape[0]])
y = p99["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# 99 Q")
print(lm1.summary())


# 99 CAPM
                                 OLS Regression Results                                
Dep. Variable:              arbitrage   R-squared (uncentered):                   0.010
Model:                            OLS   Adj. R-squared (uncentered):              0.009
Method:                 Least Squares   F-statistic:                              10.98
Date:                Tue, 12 May 2020   Prob (F-statistic):                    0.000952
Time:                        22:35:59   Log-Likelihood:                         -4286.7
No. Observations:                1114   AIC:                                      8575.
Df Residuals:                    1113   BIC:                                      8580.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

# Question 3

In [380]:
# load data
data = pd.read_excel(path+"/25_Portfolios_5x5.xlsx")
name = data.iloc[14,:].to_list()
new_data = pd.DataFrame(data.iloc[15:1139,:].values,columns=name)
new_data.head()

,NaN,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
0,192607,3.7782,-0.4119,-1.9434,0.353,2.0534,2.1904,2.4192,0.4926,-1.577,...,1.5893,1.5278,1.1869,0.2727,2.4678,3.4539,6.0902,2.0266,3.1111,0.5623
1,192608,-2.2074,-8.7275,2.4404,0.6086,8.3968,2.1709,-1.1849,4.0084,0.4643,...,1.3336,3.873,2.0059,2.1706,5.3422,1.0124,4.1903,2.0131,5.4849,7.7576
2,192609,-6.2113,-0.2989,-6.1982,-1.6368,0.8649,-1.855,-1.2618,1.0829,-3.0405,...,1.0923,-0.525,-1.7314,1.4646,0.873,-1.2906,3.6538,0.095,-0.7487,-2.4284
3,192610,-8.6241,-3.7532,-5.6719,5.717,-2.5476,-1.7995,-3.2663,-5.0745,-8.045,...,-3.3361,-2.6559,-2.0316,-3.1051,-5.3525,-2.7413,-3.0071,-2.2437,-4.6719,-5.8129
4,192611,3.4744,6.6476,2.2634,-4.702,0.5362,2.9051,-2.369,3.0078,4.6649,...,3.4448,2.3887,3.7403,4.932,1.8213,4.2946,2.5326,1.5204,3.6619,2.5636


In [396]:
# use the function from question 1
portfolio_pool_5 = get_portfolio_pool(new_data.iloc[:,1:],5,3)
portfolio_pool_5

def short_long(month_return, portfolio_pool, K, skip=1):
    index, end = portfolio_pool.index[1+skip:], portfolio_pool.index[-1]
    raw_df = pd.DataFrame(np.zeros((len(index),3)),index=index,columns=["long","short","arbitrage"])
    for i in index:
        hold_index = list(range(i,min(end,i+K)))
        for i in hold_index:
            long = portfolio_pool.loc[i,["long_0","long_1","long_2"]].values
            short = portfolio_pool.loc[i,["short_3","short_2","short_1"]].values
            #print(company_columns)
            raw_df.loc[i,"long"] += month_return.loc[i,long].sum()
            raw_df.loc[i,"short"] += month_return.loc[i,short].sum()
    raw_df["long"] /= 60
    raw_df["short"] /= 60
    raw_df["arbitrage"] = raw_df["short"] - raw_df["long"] 
    return raw_df

q3 = long_short(new_data,portfolio_pool_5,5)

 It is significant greater than zero

In [398]:
print(f"df: {q3.shape[0]}")
print(f"95%t-score: {1.646}")
print(f't66: {q3["arbitrage"].mean()/q3["arbitrage"].std()*np.sqrt(q3.shape[0])}')

df: 1118
95%t-score: 1.646
t66: 19.81441972208044


According to the t-value
Except for Rm-Rf, RMW in 5 factor model, and HXZ 1-factor mode are not significant, all other parameters are significant

In [399]:
# imports
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


# q3 CAPM
length = np.min([q3.shape[0],CAPM.shape[0]])
y = q3["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = CAPM.iloc[-length:,:].reset_index().drop("index",axis=1)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# q3 CAPM")
print(lm1.summary())


# q3 F3
length = np.min([q3.shape[0],F3.shape[0]])
y = q3["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F3.iloc[-length:,1:4].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# q3 F3")
print(lm1.summary())

# q3 F5
length = np.min([q3.shape[0],F5.shape[0]])
y = q3["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# q3 F5")
print(lm1.summary())


# q3 Q
length = np.min([q3.shape[0],Q.shape[0]])
y = q3["arbitrage"].iloc[-length:].reset_index().drop("index",axis=1)
x = F5.iloc[-length:,1:6].reset_index().drop("index",axis=1)
#print(x)
lm1 = sm.OLS(y,x.astype(float)).fit()
print("\n# q3 Q")
print(lm1.summary())


# q3 CAPM
                                 OLS Regression Results                                
Dep. Variable:              arbitrage   R-squared (uncentered):                   0.049
Model:                            OLS   Adj. R-squared (uncentered):              0.048
Method:                 Least Squares   F-statistic:                              57.63
Date:                Tue, 12 May 2020   Prob (F-statistic):                    6.65e-14
Time:                        23:20:30   Log-Likelihood:                         -3613.1
No. Observations:                1118   AIC:                                      7228.
Df Residuals:                    1117   BIC:                                      7233.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

# Question 4

In [435]:
# load data
Q4_data = pd.read_excel(path+"/book_market.xlsx",header =None)
mkt_val = pd.read_csv(path+"/ME.csv")
mkt_val.set_index("Date",inplace=True)
book_val = pd.read_csv(path+"/Book_value.csv")
book_val.set_index("Date",inplace=True)
risk_free_returns = pd.read_csv(path+"/t_bill_returns.csv")
risk_free_returns.set_index("date",inplace=True)
stock_ret = pd.read_csv(path+"/HPR_data.csv")
stock_ret.set_index("date",inplace=True)
mrkt_ret = pd.read_csv(path+"/mrkt_returns.csv")
mrkt_ret.set_index("date",inplace=True)

In [436]:
import numpy as np
stock_ret = stock_ret.apply(lambda x: np.log(x/100 +1)*100, axis=1,result_type='broadcast')
stock_ret = stock_ret.iloc[-179:]
ln_book = book_val.apply(lambda x: np.log(x))
ln_mkt = mkt_val.apply(lambda y: np.log(y))
bk_mkt_ratio = np.subtract(ln_book,ln_mkt)      
bk_mkt_ratio.head()
long_ret = []
short_ret = []
arb_ret = [] 

for i in range(len(stock_ret)):
    if i > 59 and i+1 < len(stock_ret.index): 
        expected_returns = {}
        
        for n in stock_ret.columns:
             
            BV_MV_lagged = bk_mkt_ratio[n].iloc[0:i].values 
            X = np.column_stack((np.repeat(1, len(BV_MV_lagged)),BV_MV_lagged))
            Y = stock_ret[n].iloc[0:i].values
            
            estimates =  np.matmul( np.matmul(np.linalg.inv( np.matmul(np.transpose(X), X) ) , np.transpose(X)), Y)
            expected_returns[n] = estimates[0] + estimates[1] * bk_mkt_ratio[n].values[i]
        
        sorted_data = sorted(expected_returns.items(),key=lambda x:x[1],reverse=True)
        highest = sorted_data[0:3]
        lowest = sorted_data[-3:]
        unzipped_highest = zip(*highest)
        unzipped_lowest = zip(*lowest)
        high_three =list(unzipped_highest)[0]
        low_three = list(unzipped_lowest)[0]

        net_long = 0
        net_short = 0
        for j in high_three:
            net_long += stock_ret[j].values[i+1]*(1/3)
        for k in low_three:
            net_short += stock_ret[k].values[i+1]*(1/3)
        
        long_ret.append(net_long)
        short_ret.append(net_short)
        arb_ret.append(net_long - net_short)

In [430]:
# compute t-status
print("arbitrage:")
print(f"return: {np.array(arb_ret).mean()}")
print(f"t-status: {np.array(arb_ret).mean()/np.array(arb_ret).std()}")

print("short:")
print(f"return: {np.array(short_ret).mean()}")
print(f"t-status: {np.array(short_ret).mean()/np.array(long_ret).std()}")

print("long:")
print(f"return: {np.array(long_ret).mean()}")
print(f"t-status: {np.array(long_ret).mean()/np.array(long_ret).std()}")

arbitrage:
return: 0.0020941903582484445
t-status: 0.04856800066021095
short:
return: 0.01569530610190985
t-status: 0.3494865082381052
long:
return: 0.0177894964601583
t-status: 0.39611772849835214


# question 5

In [438]:
# load data
stock_ret_no_div = pd.read_csv(path+"/HPR_data_wo_div.csv")
stock_ret_no_div.set_index("date",inplace=True)

log_stock_no_div = stock_ret_no_div.apply(lambda x:np.log(x/100 +1)*100, axis=1,result_type='broadcast')
log_stock_no_div = log_stock_no_div.iloc[-179:]
dp_ratio=np.subtract(stock_ret , log_stock_no_div)


In [439]:
long_ret = []
short_ret = []
arb_ret = [] 

for i in range(len(stock_ret)):
    if i > 59 and i < len(stock_ret.index)-1: 
        expected_returns = {}
        
        for n in stock_ret.columns:
             
            DP_ratio = dp_ratio[n].iloc[0:i].values 
            X = np.column_stack((np.repeat(1, len(DP_ratio)),DP_ratio))
            Y = stock_ret[n].iloc[0:i].values
            
            estimates =  np.matmul( np.matmul(np.linalg.inv( np.matmul(np.transpose(X), X) ) , np.transpose(X)), Y)
            expected_returns[n] = estimates[0] + estimates[1] * bk_mkt_ratio[n].values[i]
        
        sorted_data = sorted(expected_returns.items(),key=lambda x:x[1],reverse=True)
        highest = sorted_data[0:3]
        lowest = sorted_data[-3:]
        unzipped_highest = zip(*highest)
        unzipped_lowest = zip(*lowest)
        high_three =list(unzipped_highest)[0]
        low_three = list(unzipped_lowest)[0]

        net_long = 0
        net_short = 0
        for j in high_three:
            net_long += stock_ret[j].values[i+1]*(1/3)
        for k in low_three:
            net_short += stock_ret[k].values[i+1]*(1/3)
        
        long_ret.append(net_long)
        short_ret.append(net_short)
        arb_ret.append(net_long - net_short)

In [440]:
# compute t-status
print("arbitrage:")
print(f"return: {np.array(arb_ret).mean()}")
print(f"t-status: {np.array(arb_ret).mean()/np.array(arb_ret).std()}")

print("short:")
print(f"return: {np.array(short_ret).mean()}")
print(f"t-status: {np.array(short_ret).mean()/np.array(long_ret).std()}")

print("long:")
print(f"return: {np.array(long_ret).mean()}")
print(f"t-status: {np.array(long_ret).mean()/np.array(long_ret).std()}")

arbitrage:
return: 0.0016691245415136299
t-status: 0.03838523936983855
short:
return: 0.01635686839374341
t-status: 0.36376643149168636
long:
return: 0.018025992935257036
t-status: 0.40088670803642246
